This notebook tests on applying bagging over small convolutional networks.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, utils

print(tf.__version__) # 2.10.0

utils.set_random_seed(812)

2024-05-03 04:10:24.001146: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 04:10:24.147259: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2.10.0


GPU Setup

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-05-03 04:10:27.688026: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 04:10:27.807701: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 04:10:27.808045: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPUs found. Running on CPU.")

1 Physical GPUs, 1 Logical GPUs


2024-05-03 04:10:37.045280: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 04:10:37.048732: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 04:10:37.048782: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-03 04:10:37.048796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kerne

Dataset used: MNIST, Fashion MNIST, CIFAR-10. Datasets are normalized so that the values lie between 0 and 1.

In [5]:
(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = datasets.mnist.load_data()
mnist_train_images = mnist_train_images / 255.0
mnist_test_images = mnist_test_images / 255.0
print(mnist_train_images.shape, mnist_train_labels.shape, mnist_test_images.shape, mnist_test_labels.shape)

(fmnist_train_images, fmnist_train_labels), (fmnist_test_images, fmnist_test_labels) = datasets.fashion_mnist.load_data()
fmnist_train_images = fmnist_train_images / 255.0
fmnist_test_images = fmnist_test_images / 255.0
print(fmnist_train_images.shape, fmnist_test_images.shape, fmnist_train_labels.shape, fmnist_test_labels.shape)


(cifar_train_images, cifar_train_labels), (cifar_test_images, cifar_test_labels) = datasets.cifar10.load_data()
cifar_train_images = cifar_train_images / 255.0
cifar_test_images = cifar_test_images / 255.0
print(cifar_train_images.shape, cifar_train_labels.shape, cifar_test_images.shape, cifar_test_labels.shape)



(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)
(60000, 28, 28) (10000, 28, 28) (60000,) (10000,)
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [13]:
print(type(cifar_train_labels))
print(cifar_test_labels[:-5])

<class 'numpy.ndarray'>
[[3]
 [8]
 [8]
 ...
 [3]
 [5]
 [3]]


# Training the base learner
The following functions outputs a base CNN model. 

For MNIST and Fashion MNIST, there is 
- one convolutional layer with 32 filters 
- followed by a max-pooling layer and a dense layer 

For CIFAR-10, there are two convolutional layer, each followed by a max-pooling layer, and a dense layer at last. The models are trained through SGD with cross-entropy loss. 

Early-stopping is applied based on validation accuracy. <br>
The output of these CNNs are logits instead of probabilities.

In [7]:
def train_base_model(dataset_name, train_images, train_labels):
    model = models.Sequential()
    if dataset_name == 'mnist':
        #First Layer
        model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=(28, 28, 1)))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        #Second layer
        model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=(28, 28, 1)))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        #Flatten and dense layer
        model.add(layers.Flatten())
        model.add(layers.Dense(units=10))
        model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.05),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]) 
        model.fit(train_images, train_labels, epochs=50,
              validation_split=0.2,
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=3, min_delta=0.001)], 
              batch_size=75, 
              verbose=2) # Default batch size is 32
    elif dataset_name == 'fashion_mnist':
        model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=(28, 28, 1)))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=(28, 28, 1)))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dense(units=10))
        model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.05),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]) 
        model.fit(train_images, train_labels, epochs=50,
              validation_split=0.2,
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=3, min_delta=0.001)], 
              batch_size=75, 
              verbose=2) 
    elif dataset_name == 'cifar10':
        #First Layer
        model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=(32, 32, 3)))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        #Second Layer
        model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='valid'))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        
        #Third Layer
        model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid'))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        #Flatten and Dense layer
        model.add(layers.Flatten())
        model.add(layers.Dense(units=10))
        model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.05),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]) 
        model.fit(train_images, train_labels, epochs=50,
              validation_split=0.2,
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=3, min_delta=0.001)], 
              batch_size=75, 
              verbose=2) 
    return model

# Training bagging models
`bagging_train()` trains a bagging model with `num_models` base learners using `num_examples` random examples from the training set. The resulting model would be a list of CNN networks. `bagging_test()` tests the model and returns the training and test losses as well as accuracies. <br>
The bagging model makes a prediction by taking the average of the logits from each base learners.

In [9]:
def bagging_train(num_models, num_examples, train_images, train_labels, dataset_name):
    
    bagging_model = []
    
    for i in range(num_models):
        # Generate a subset of training examples
        indices = np.random.choice(train_images.shape[0], num_examples, replace=False)
        train_images_subset = train_images[indices]
        train_labels_subset = train_labels[indices]
        
        base_model = train_base_model(dataset_name, train_images_subset, train_labels_subset)
        bagging_model.append(base_model)
        
    return bagging_model

def bagging_test(bagging_model, train_images, train_labels, test_images, test_labels):
    train_predictions = np.empty(len(bagging_model), dtype=object)
    test_predictions = np.empty(len(bagging_model), dtype=object)
        
    for i, base_model in enumerate(bagging_model):
        train_predictions[i] = base_model.predict(train_images, verbose=2)
        test_predictions[i] = base_model.predict(test_images, verbose=2)
        
    train_output = np.mean(train_predictions, axis=0)
    test_output = np.mean(test_predictions, axis=0)
    
    cross_entropy_tr = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    train_loss = cross_entropy_tr(train_labels, train_output).numpy()
    categorical_accuracy_tr = tf.keras.metrics.SparseCategoricalAccuracy()
    categorical_accuracy_tr.update_state(train_labels, train_output)
    train_accuracy = categorical_accuracy_tr.result().numpy()
    
    cross_entropy_te = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    test_loss = cross_entropy_te(test_labels, test_output).numpy()
    categorical_accuracy_te = tf.keras.metrics.SparseCategoricalAccuracy()
    categorical_accuracy_te.update_state(test_labels, test_output)
    test_accuracy = categorical_accuracy_te.result().numpy()
    
    return train_loss, train_accuracy, test_loss, test_accuracy



For each of the following experiments, a base CNN model and a bagging model is trained; The bagging model have 5 base model, each trained with half of the training set.

# MNIST


In [8]:
train_images = mnist_train_images
train_labels = mnist_train_labels
test_images = mnist_test_images
test_labels = mnist_test_labels


cnn_model = train_base_model('mnist', train_images, train_labels)

train_loss, train_accuracy = cnn_model.evaluate(train_images, train_labels, verbose=2) 
test_loss, test_accuracy = cnn_model.evaluate(test_images, test_labels, verbose=2)
print("Training loss:", train_loss)
print("Test loss:", test_loss)
print("Training accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)



Epoch 1/50
640/640 - 1s - loss: 0.4765 - sparse_categorical_accuracy: 0.8631 - val_loss: 0.3093 - val_sparse_categorical_accuracy: 0.9112 - 1s/epoch - 2ms/step
Epoch 2/50
640/640 - 1s - loss: 0.2957 - sparse_categorical_accuracy: 0.9133 - val_loss: 0.2450 - val_sparse_categorical_accuracy: 0.9300 - 1s/epoch - 2ms/step
Epoch 3/50
640/640 - 1s - loss: 0.2367 - sparse_categorical_accuracy: 0.9314 - val_loss: 0.2057 - val_sparse_categorical_accuracy: 0.9427 - 1s/epoch - 2ms/step
Epoch 4/50
640/640 - 1s - loss: 0.1869 - sparse_categorical_accuracy: 0.9470 - val_loss: 0.1525 - val_sparse_categorical_accuracy: 0.9602 - 1s/epoch - 2ms/step
Epoch 5/50
640/640 - 1s - loss: 0.1495 - sparse_categorical_accuracy: 0.9577 - val_loss: 0.1310 - val_sparse_categorical_accuracy: 0.9647 - 1s/epoch - 2ms/step
Epoch 6/50
640/640 - 1s - loss: 0.1248 - sparse_categorical_accuracy: 0.9655 - val_loss: 0.1178 - val_sparse_categorical_accuracy: 0.9682 - 1s/epoch - 2ms/step
Epoch 7/50
640/640 - 1s - loss: 0.1079 -

With GPU

In [28]:
train_images = mnist_train_images
train_labels = mnist_train_labels
test_images = mnist_test_images
test_labels = mnist_test_labels

with tf.device('/GPU:0'):
    cnn_model = train_base_model('mnist', train_images, train_labels)

    train_loss, train_accuracy = cnn_model.evaluate(train_images, train_labels, verbose=2) 
    test_loss, test_accuracy = cnn_model.evaluate(test_images, test_labels, verbose=2)

print("Training loss:", train_loss)
print("Test loss:", test_loss)
print("Training accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)



Epoch 1/50
640/640 - 3s - loss: 0.4610 - sparse_categorical_accuracy: 0.8619 - val_loss: 0.1448 - val_sparse_categorical_accuracy: 0.9592 - 3s/epoch - 5ms/step
Epoch 2/50
640/640 - 2s - loss: 0.1250 - sparse_categorical_accuracy: 0.9619 - val_loss: 0.1044 - val_sparse_categorical_accuracy: 0.9683 - 2s/epoch - 2ms/step
Epoch 3/50
640/640 - 2s - loss: 0.0930 - sparse_categorical_accuracy: 0.9715 - val_loss: 0.0807 - val_sparse_categorical_accuracy: 0.9753 - 2s/epoch - 2ms/step
Epoch 4/50
640/640 - 2s - loss: 0.0785 - sparse_categorical_accuracy: 0.9759 - val_loss: 0.0707 - val_sparse_categorical_accuracy: 0.9788 - 2s/epoch - 2ms/step
Epoch 5/50
640/640 - 2s - loss: 0.0688 - sparse_categorical_accuracy: 0.9792 - val_loss: 0.0695 - val_sparse_categorical_accuracy: 0.9793 - 2s/epoch - 2ms/step
Epoch 6/50
640/640 - 2s - loss: 0.0608 - sparse_categorical_accuracy: 0.9811 - val_loss: 0.0738 - val_sparse_categorical_accuracy: 0.9774 - 2s/epoch - 2ms/step
Epoch 7/50
640/640 - 2s - loss: 0.0560 -

In [6]:
bagging_model = bagging_train(5, 30000, train_images, train_labels, 'mnist')
train_loss, train_accuracy, test_loss, test_accuracy = bagging_test(bagging_model, train_images, train_labels, test_images, test_labels)

print("Training loss:", train_loss)
print("Test loss:", test_loss)
print("Training accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)


Epoch 1/50
320/320 - 2s - loss: 0.5978 - sparse_categorical_accuracy: 0.8290 - val_loss: 0.3628 - val_sparse_categorical_accuracy: 0.8927 - 2s/epoch - 8ms/step
Epoch 2/50
320/320 - 2s - loss: 0.3337 - sparse_categorical_accuracy: 0.9008 - val_loss: 0.3339 - val_sparse_categorical_accuracy: 0.9003 - 2s/epoch - 7ms/step
Epoch 3/50
320/320 - 4s - loss: 0.2990 - sparse_categorical_accuracy: 0.9118 - val_loss: 0.3043 - val_sparse_categorical_accuracy: 0.9127 - 4s/epoch - 12ms/step
Epoch 4/50
320/320 - 1s - loss: 0.2718 - sparse_categorical_accuracy: 0.9207 - val_loss: 0.2866 - val_sparse_categorical_accuracy: 0.9123 - 1s/epoch - 4ms/step
Epoch 5/50
320/320 - 1s - loss: 0.2439 - sparse_categorical_accuracy: 0.9290 - val_loss: 0.2504 - val_sparse_categorical_accuracy: 0.9242 - 1s/epoch - 4ms/step
Epoch 6/50
320/320 - 1s - loss: 0.2151 - sparse_categorical_accuracy: 0.9393 - val_loss: 0.2292 - val_sparse_categorical_accuracy: 0.9320 - 1s/epoch - 4ms/step
Epoch 7/50
320/320 - 1s - loss: 0.1899 

In [17]:
train_images = mnist_train_images
train_labels = mnist_train_labels
test_images = mnist_test_images
test_labels = mnist_test_labels

with tf.device('GPU:0'):
    bagging_model = bagging_train(5, 30000, train_images, train_labels, 'mnist')
    train_loss, train_accuracy, test_loss, test_accuracy = bagging_test(bagging_model, train_images, train_labels, test_images, test_labels)

print("Training loss:", train_loss)
print("Test loss:", test_loss)
print("Training accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)


Epoch 1/50
320/320 - 1s - loss: 0.5971 - sparse_categorical_accuracy: 0.8317 - val_loss: 0.3477 - val_sparse_categorical_accuracy: 0.8957 - 1s/epoch - 4ms/step
Epoch 2/50
320/320 - 1s - loss: 0.3399 - sparse_categorical_accuracy: 0.9003 - val_loss: 0.3150 - val_sparse_categorical_accuracy: 0.9027 - 721ms/epoch - 2ms/step
Epoch 3/50
320/320 - 1s - loss: 0.3043 - sparse_categorical_accuracy: 0.9104 - val_loss: 0.2751 - val_sparse_categorical_accuracy: 0.9173 - 727ms/epoch - 2ms/step
Epoch 4/50
320/320 - 1s - loss: 0.2733 - sparse_categorical_accuracy: 0.9202 - val_loss: 0.2578 - val_sparse_categorical_accuracy: 0.9192 - 770ms/epoch - 2ms/step
Epoch 5/50
320/320 - 1s - loss: 0.2468 - sparse_categorical_accuracy: 0.9290 - val_loss: 0.2410 - val_sparse_categorical_accuracy: 0.9262 - 733ms/epoch - 2ms/step
Epoch 6/50
320/320 - 1s - loss: 0.2161 - sparse_categorical_accuracy: 0.9386 - val_loss: 0.2130 - val_sparse_categorical_accuracy: 0.9360 - 749ms/epoch - 2ms/step
Epoch 7/50
320/320 - 1s -

# Fashion MNIST

In [7]:
train_images = fmnist_train_images
train_labels = fmnist_train_labels
test_images = fmnist_test_images
test_labels = fmnist_test_labels


cnn_model = train_base_model('fashion_mnist', train_images, train_labels)

train_loss, train_accuracy = cnn_model.evaluate(train_images, train_labels, verbose=2)
test_loss, test_accuracy = cnn_model.evaluate(test_images, test_labels, verbose=2)
print("Training loss:", train_loss)
print("Test loss:", test_loss)
print("Training accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)



Epoch 1/50
640/640 - 3s - loss: 0.7117 - sparse_categorical_accuracy: 0.7545 - val_loss: 0.5111 - val_sparse_categorical_accuracy: 0.8203 - 3s/epoch - 5ms/step
Epoch 2/50
640/640 - 3s - loss: 0.4680 - sparse_categorical_accuracy: 0.8365 - val_loss: 0.4140 - val_sparse_categorical_accuracy: 0.8548 - 3s/epoch - 5ms/step
Epoch 3/50
640/640 - 3s - loss: 0.4088 - sparse_categorical_accuracy: 0.8584 - val_loss: 0.4030 - val_sparse_categorical_accuracy: 0.8577 - 3s/epoch - 4ms/step
Epoch 4/50
640/640 - 3s - loss: 0.3746 - sparse_categorical_accuracy: 0.8704 - val_loss: 0.3682 - val_sparse_categorical_accuracy: 0.8706 - 3s/epoch - 4ms/step
Epoch 5/50
640/640 - 3s - loss: 0.3523 - sparse_categorical_accuracy: 0.8788 - val_loss: 0.3522 - val_sparse_categorical_accuracy: 0.8769 - 3s/epoch - 4ms/step
Epoch 6/50
640/640 - 3s - loss: 0.3363 - sparse_categorical_accuracy: 0.8836 - val_loss: 0.3586 - val_sparse_categorical_accuracy: 0.8722 - 3s/epoch - 4ms/step
Epoch 7/50
640/640 - 3s - loss: 0.3228 -

In [8]:
bagging_model = bagging_train(5, 30000, train_images, train_labels, 'fashion_mnist')
train_loss, train_accuracy, test_loss, test_accuracy = bagging_test(bagging_model, train_images, train_labels, test_images, test_labels)
print("Training loss:", train_loss)
print("Test loss:", test_loss)
print("Training accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)


Epoch 1/50
320/320 - 2s - loss: 0.8747 - sparse_categorical_accuracy: 0.7078 - val_loss: 0.6401 - val_sparse_categorical_accuracy: 0.7768 - 2s/epoch - 5ms/step
Epoch 2/50
320/320 - 2s - loss: 0.5643 - sparse_categorical_accuracy: 0.8040 - val_loss: 0.7146 - val_sparse_categorical_accuracy: 0.7595 - 2s/epoch - 5ms/step
Epoch 3/50
320/320 - 2s - loss: 0.4908 - sparse_categorical_accuracy: 0.8278 - val_loss: 0.4740 - val_sparse_categorical_accuracy: 0.8273 - 2s/epoch - 5ms/step
Epoch 4/50
320/320 - 2s - loss: 0.4485 - sparse_categorical_accuracy: 0.8403 - val_loss: 0.4616 - val_sparse_categorical_accuracy: 0.8308 - 2s/epoch - 5ms/step
Epoch 5/50
320/320 - 2s - loss: 0.4191 - sparse_categorical_accuracy: 0.8544 - val_loss: 0.4203 - val_sparse_categorical_accuracy: 0.8460 - 2s/epoch - 5ms/step
Epoch 6/50
320/320 - 1s - loss: 0.3968 - sparse_categorical_accuracy: 0.8625 - val_loss: 0.4005 - val_sparse_categorical_accuracy: 0.8550 - 1s/epoch - 4ms/step
Epoch 7/50
320/320 - 1s - loss: 0.3797 -

# CIFAR-10

In [9]:
train_images = cifar_train_images
train_labels = cifar_train_labels
test_images = cifar_test_images
test_labels = cifar_test_labels


cnn_model = train_base_model('cifar10', train_images, train_labels)

train_loss, train_accuracy = cnn_model.evaluate(train_images, train_labels, verbose=2)
test_loss, test_accuracy = cnn_model.evaluate(test_images, test_labels, verbose=2)
print("Training loss:", train_loss)
print("Test loss:", test_loss)
print("Training accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)


Epoch 1/50
534/534 - 6s - loss: 1.9356 - sparse_categorical_accuracy: 0.3060 - val_loss: 1.6719 - val_sparse_categorical_accuracy: 0.3985 - 6s/epoch - 11ms/step
Epoch 2/50
534/534 - 5s - loss: 1.5499 - sparse_categorical_accuracy: 0.4504 - val_loss: 1.4571 - val_sparse_categorical_accuracy: 0.4892 - 5s/epoch - 9ms/step
Epoch 3/50
534/534 - 5s - loss: 1.3839 - sparse_categorical_accuracy: 0.5135 - val_loss: 1.3429 - val_sparse_categorical_accuracy: 0.5307 - 5s/epoch - 9ms/step
Epoch 4/50
534/534 - 5s - loss: 1.2699 - sparse_categorical_accuracy: 0.5549 - val_loss: 1.2769 - val_sparse_categorical_accuracy: 0.5466 - 5s/epoch - 9ms/step
Epoch 5/50
534/534 - 5s - loss: 1.1892 - sparse_categorical_accuracy: 0.5832 - val_loss: 1.2508 - val_sparse_categorical_accuracy: 0.5614 - 5s/epoch - 9ms/step
Epoch 6/50
534/534 - 5s - loss: 1.1236 - sparse_categorical_accuracy: 0.6115 - val_loss: 1.4848 - val_sparse_categorical_accuracy: 0.4728 - 5s/epoch - 9ms/step
Epoch 7/50
534/534 - 5s - loss: 1.0752 

With GPU

In [10]:
train_images = cifar_train_images
train_labels = cifar_train_labels
test_images = cifar_test_images
test_labels = cifar_test_labels

with tf.device('GPU:0'):
    cnn_model = train_base_model('cifar10', train_images, train_labels)

    train_loss, train_accuracy = cnn_model.evaluate(train_images, train_labels, verbose=2)
    test_loss, test_accuracy = cnn_model.evaluate(test_images, test_labels, verbose=2)

print("Training loss:", train_loss)
print("Test loss:", test_loss)
print("Training accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)


Epoch 1/50


2024-05-03 04:11:27.262948: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8907
2024-05-03 04:11:27.445177: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-05-03 04:11:27.445217: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2024-05-03 04:11:27.445369: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-05-03 04:11:27.608684: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


534/534 - 3s - loss: 2.0547 - sparse_categorical_accuracy: 0.2491 - val_loss: 1.8000 - val_sparse_categorical_accuracy: 0.3487 - 3s/epoch - 6ms/step
Epoch 2/50
534/534 - 2s - loss: 1.6779 - sparse_categorical_accuracy: 0.3981 - val_loss: 1.5771 - val_sparse_categorical_accuracy: 0.4370 - 2s/epoch - 4ms/step
Epoch 3/50
534/534 - 2s - loss: 1.4851 - sparse_categorical_accuracy: 0.4697 - val_loss: 1.4556 - val_sparse_categorical_accuracy: 0.4690 - 2s/epoch - 4ms/step
Epoch 4/50
534/534 - 2s - loss: 1.3611 - sparse_categorical_accuracy: 0.5160 - val_loss: 1.4733 - val_sparse_categorical_accuracy: 0.4658 - 2s/epoch - 4ms/step
Epoch 5/50
534/534 - 2s - loss: 1.2695 - sparse_categorical_accuracy: 0.5509 - val_loss: 1.3428 - val_sparse_categorical_accuracy: 0.5197 - 2s/epoch - 4ms/step
Epoch 6/50
534/534 - 2s - loss: 1.1901 - sparse_categorical_accuracy: 0.5820 - val_loss: 1.5164 - val_sparse_categorical_accuracy: 0.4634 - 2s/epoch - 4ms/step
Epoch 7/50
534/534 - 2s - loss: 1.1268 - sparse_cat

In [11]:
with tf.device('GPU:0'):
    bagging_model = bagging_train(5, 25000, train_images, train_labels, 'cifar10')
    train_loss, train_accuracy, test_loss, test_accuracy = bagging_test(bagging_model, train_images, train_labels, test_images, test_labels)
print("Training loss:", train_loss)
print("Test loss:", test_loss)
print("Training accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)


Epoch 1/50
267/267 - 2s - loss: 2.1653 - sparse_categorical_accuracy: 0.1957 - val_loss: 2.0479 - val_sparse_categorical_accuracy: 0.2532 - 2s/epoch - 8ms/step
Epoch 2/50
267/267 - 1s - loss: 1.9425 - sparse_categorical_accuracy: 0.2950 - val_loss: 1.7977 - val_sparse_categorical_accuracy: 0.3630 - 1s/epoch - 4ms/step
Epoch 3/50
267/267 - 1s - loss: 1.7497 - sparse_categorical_accuracy: 0.3685 - val_loss: 1.7023 - val_sparse_categorical_accuracy: 0.3974 - 987ms/epoch - 4ms/step
Epoch 4/50
267/267 - 1s - loss: 1.6114 - sparse_categorical_accuracy: 0.4228 - val_loss: 1.6191 - val_sparse_categorical_accuracy: 0.4284 - 982ms/epoch - 4ms/step
Epoch 5/50
267/267 - 1s - loss: 1.5130 - sparse_categorical_accuracy: 0.4575 - val_loss: 1.4909 - val_sparse_categorical_accuracy: 0.4644 - 1s/epoch - 4ms/step
Epoch 6/50
267/267 - 1s - loss: 1.4326 - sparse_categorical_accuracy: 0.4905 - val_loss: 1.3929 - val_sparse_categorical_accuracy: 0.5022 - 1s/epoch - 4ms/step
Epoch 7/50
267/267 - 1s - loss: 1.

: 